# Numerical Optimization (CS215300) Assignment 3
## Introduction
In this assignment, we expect you to be able to solve constrained optimization problem by Scipy library. We want you to apply two algorithms on the following problem as benchmark, survey the method used in these libraries, and analysis the behaviour of these algorithms.
The library document link: https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html

## Task
1. Please solve the following problrem by using `trust-constr` method:

$$
        \begin{array}{cc}
                \displaystyle \min_{x_1,x_2} & f(x_1,x_2)=-x_1-x_2 \\
        \end{array}
$$

$$
        \text{s.t.} -2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2 \le 0  \\
        -4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36 \le 0   \\
        0 \le x_1 \le 3 \\
        0 \le x_2 \le 4 \\
$$

2. Please use COBYLA method to solve the same problem.
3. In your report, please read the paper cited in the libraries, which gives the details of the algorithms. Write a brief introduction of the algorithms, and compare their behaviours in this benchmark. You are not necessarily to read the original paper, other resourses (ex. slides from other schools or surveys) are also acceptable, please include the link or paper name in your reference if you referred other resources.
4. Please provide the Jacobian and Hessian function in matrix form in your report. Basic latex syntax is supported in Markdown.
5. Rename this notebook file with your student ID and upload it to eeclass platform. (ex. 107xxxxxx.ipynb)

In [12]:
import numpy as np
from scipy.optimize import Bounds
from scipy.optimize import minimize
from scipy.optimize import NonlinearConstraint

### Define objective function

In [13]:
def f(x):
    return - x[0] - x[1]

def grad(x):
    return np.array([-1.0, -1.0])

def hess(x):
    return np.array([[0.0,0.0],[0.0,0.0]])

### Define constraint functions and derivatives

In [14]:
def cons_f(x):
    constrain1 = (-2.0*x[0]**4) +  (8.0*x[0]**3) -  (8.0*x[0]**2) + x[1] - 2.0
    constrain2 = (-4.0*x[0]**4) + (32.0*x[0]**3) - (88.0*x[0]**2) + (96.0*x[0]) + x[1] - 36.0
    return [constrain1, constrain2]

def cons_Jacobian(x):
    jacobian1 = [(-8.0*(x[0]**3)) + (24.0*(x[0]**2)) - (16.0*x[0]), 1.0]
    jacobian2 = [(-16.0*(x[0]**3)) + (96.0*(x[0]**2)) - (176.0*x[0]) + 96.0, 1.0]
    return [jacobian1, jacobian2]

def cons_Hessian(x, v):
    hessian1 = np.array([[(-24.0*x[0]**2) +  (48.0*x[0]) -  16.0, 0.0], [0.0, 0.0]])
    hessian2 = np.array([[(-48.0*x[0]**2) + (192.0*x[0]) - 176.0, 0.0], [0.0, 0.0]])
    return v[0]*hessian1 + v[1]*hessian2

nonlinear_constraint = NonlinearConstraint(
    cons_f,  
    -np.inf,
    0,
    jac = cons_Jacobian, 
    hess = cons_Hessian
)

### Call the minimize library

In [15]:
bounds = Bounds([0, 0], [3.0, 4.0])
x0 = [0,0]
res = minimize(
    f, 
    x0, 
    method = 'trust-constr', 
    jac = grad, 
    hess = hess, 
    constraints = [nonlinear_constraint], 
    options = {'verbose': 1},
    bounds = bounds
)
print(res.x)

`gtol` termination condition is satisfied.
Number of iterations: 17, function evaluations: 13, CG iterations: 12, optimality: 7.68e-09, constraint violation: 0.00e+00, execution time: 0.031 s.
[0.61157004 3.44188615]


### Apply COBYLA method

Since 'COBYLA' requires constraints to be defined as a sequence of dictionaries, with `type`, `fun` and `jac`, we have to redefined our inequality contraints.

In [16]:
ineq_cons = {
    'type': 'ineq',
    'fun' : lambda x: np.array(
        (-2.0*x[0]**4) +  (8.0*x[0]**3) -  (8.0*x[0]**2) + x[1] - 2.0,  
        (-4.0*x[0]**4) + (32.0*x[0]**3) - (88.0*x[0]**2) + (96.0*x[0]) + x[1] - 36.0
    ),
    'jac' : lambda x: np.array(
        (-8.0*(x[0]**3)) + (24.0*(x[0]**2)) - (16.0*x[0]), 1.0,
        (-16.0*(x[0]**3)) + (96.0*(x[0]**2)) - (176.0*x[0]) + 96.0, 1.0
    )
}

x0 = [0,0]
res = minimize(
    f, 
    x0, 
    method = 'cobyla', 
    constraints = [ineq_cons],
    options={'disp': True}
)
print(res.x)

[  4.34481012 208.81549884]
   Return from subroutine COBYLA because rounding errors are becoming damaging.

   NFVALS =  417   F =-2.131603E+02    MAXCV = 7.651946E-01
   X = 4.344810E+00   2.088155E+02



# Report

### 1. Trust Region Method

The result that I got from `trust-constr` is `[0.61157004 3.44188615]` where the initial guess is `[0,0]`.

Before discussing how Trust Region Method, or TRM, works, first we have to define what is a trust region. Trust region is a subset of region of an objective function that is acquired from a quadratic function. 

It works in a way that first define a region around the current best solution, in which a certain model (usually a quadratic model) can to some extent approximate the original objective function. TRM then take a step forward according to the model depicts within the region. Unlike the line search methods, TRM usually determines the step size before the improving direction (or at the same time). If a notable decrease (our following discussion will based on minimization problems) is gained after the step forward, then the model is believed to be a good representation of the original objective function. If the improvement is too subtle or even a negative improvement is gained, then the model is not to be believed as a good representation of the original objective function within that region. The convergence can be ensured that the size of the “trust region” (usually defined by the radius in Euclidean norm) in each iteration would depend on the improvement previously made. 

### 2. COBYLA Method

The result that I got from `cobyla` is `[4.34481012 208.81549884]`.

Unlike `trust-constr`, Constrained optimization by linear approximation, or COBYLA, is a numerical optimization algorithm for constrained problems where the derivative of the objective function is not known.

It works by iteratively approximating the actual constrained optimization problem with linear programming problems. During an iteration, an approximating linear programming problem is solved to obtain a candidate for the optimal solution. The candidate solution is evaluated using the original objective and constraint functions, yielding a new data point in the optimization space. This information is used to improve the approximating linear programming problem used for the next iteration of the algorithm. When the solution cannot be improved anymore, the step size is reduced, refining the search. When the step size becomes sufficiently small, the algorithm finishes.

# Functions & Matrices Needed

### The Objective Function

$$
    f(x_1, x_2) = -x_1 - x_2
$$

### The Gradient Objective Function

$$
    \nabla f(x_1, x_2) = \begin{bmatrix} -1 & -1 \end{bmatrix}
$$

### The Hessian Objective Function

$$
    \nabla^2 f(x_1, x_2) = \begin{bmatrix} 0 & 0 \\ 0 & 0 \end{bmatrix}
$$

### The Constraints

$$
    \begin{bmatrix}
        -2x_1^4 + 8x_1^3 -8x_1^2 + x_2 - 2  \\
        -4x_1^4 + 32x_1^3 - 88x_1^2 + 96x_1 + x_2 -36
    \end{bmatrix}
$$

### The Constraint Jacobian Matrix

$$
    \nabla J(x) = \begin{bmatrix}
        -8x_1^3 + 24x_1 - 16 x_1 && 1 \\
        -16x_1^3 + 96x_1^2 - 176x_1 + 96 && 1
    \end{bmatrix}
$$

### The Constraint Hessian Matrix

$$
    \nabla H(x, v) = v_0 \begin{bmatrix}
        -24x_1^2 + 8 & 0 \\ 0 & 0
    \end{bmatrix} + v_1 \begin{bmatrix}
        -48x_1^2 + 192x_1 - 176 & 0 \\ 0 & 0
    \end{bmatrix}
$$

# References

* [https://optimization.mccormick.northwestern.edu/index.php/Trust-region_methods](https://optimization.mccormick.northwestern.edu/index.php/Trust-region_methods)
* [https://en.wikipedia.org/wiki/Trust_region](https://en.wikipedia.org/wiki/Trust_region)
* [https://handwiki.org/wiki/COBYLA](https://handwiki.org/wiki/COBYLA)